### Add Directories/ Load Files

In [1]:
import os
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from utils import find_folders

In [2]:
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


In [3]:
accel_blocks_df = pd.read_excel(os.path.join(
    project_path,
    'results',
    'accelerometer',
    'Accel_Blocks.xlsx'
))

accel_blocks_df.head(10)

,Percept_ID,Blocks,Cond
0,Sub005,1,1
1,Sub005,2,1
2,Sub005,5,2
3,Sub005,6,2
4,Sub005,7,3
5,Sub006,1,1
6,Sub006,2,1
7,Sub006,3,1
8,Sub006,13,2
9,Sub006,14,2


In [4]:
features_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\ENTRAINMENT\\Accelerometer\\without_med_FTG\\raw_files\\retap_results\\features'
blocks_path = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\ENTRAINMENT\\Accelerometer\\without_med_FTG\\raw_files\\retap_results\\extracted_tapblocks'

In [5]:
this_sub = 'Sub029'
this_cond = 1

these_blocks = accel_blocks_df.loc[(accel_blocks_df['Percept_ID'] == this_sub) & (accel_blocks_df['Cond'] == this_cond), 'Blocks']
block_strings = ['block' + str(num) for num in these_blocks]
print(block_strings)

['block1', 'block2', 'block3']


In [6]:
# Filter the json files based on conditions
jsons_to_import = []
for filename in os.listdir(features_path):
    if filename.endswith('.json') and this_sub in filename:
        for block_str in block_strings:
            if filename.endswith(f'{block_str}.json'):
                jsons_to_import.append(filename)

jsons_to_import

['features_Sub029_M1S0_250Hz_LHAND_left_block1.json',
 'features_Sub029_M1S0_250Hz_LHAND_left_block2.json',
 'features_Sub029_M1S0_250Hz_LHAND_left_block3.json']

In [7]:
# Filter the csv files based on conditions
csvs_to_import = []
for filename in os.listdir(blocks_path):
    if filename.endswith('.csv') and this_sub in filename:
        for block_str in block_strings:
            if filename.endswith(f'{block_str}_250Hz.csv'):
                csvs_to_import.append(filename)

csvs_to_import

['Sub029_M1S0_250Hz_LHAND_left_block1_250Hz.csv',
 'Sub029_M1S0_250Hz_LHAND_left_block2_250Hz.csv',
 'Sub029_M1S0_250Hz_LHAND_left_block3_250Hz.csv']

In [8]:
# Read the selected JSON files
imp_features = ['freq', 'mean_raise_velocity', 'trace_RMSn', 'coefVar_impactRMS', 'coefVar_intraTapInt']

combined_dict = []
for filename in jsons_to_import:
    file_path = os.path.join(features_path, filename)
    with open(file_path) as f:
        this_block_feat = json.load(f)
        matching_keys = set(imp_features) & set(this_block_feat.keys())
        combined_dict.append({key: this_block_feat[key] for key in matching_keys})


In [9]:
combined_dict

[{'coefVar_impactRMS': 0.0860521778840747,
  'coefVar_intraTapInt': 0.0402834833029361,
  'trace_RMSn': 0.1360721841905734,
  'mean_raise_velocity': 28.340883894233727,
  'freq': 2.7147876077930375},
 {'coefVar_impactRMS': 0.15903355672225367,
  'coefVar_intraTapInt': 0.4868478857745488,
  'trace_RMSn': 0.15624312742421964,
  'mean_raise_velocity': 21.45000204528345,
  'freq': 2.5},
 {'coefVar_impactRMS': 0.20412462478698437,
  'coefVar_intraTapInt': 0.2947813703260938,
  'trace_RMSn': 0.17262875851384318,
  'mean_raise_velocity': 30.487217911516044,
  'freq': 3.080826386371874}]

In [10]:
%matplotlib qt
fig, axs = plt.subplots(len(csvs_to_import),1, figsize = (18,10))

suptitle = str(this_sub) + ' - Condition ' + str(this_cond)
fig.suptitle(suptitle, fontsize=14, fontweight='bold')

rounded_list = [
    {key: round(value, 2) for key, value in dictionary.items()}
    for dictionary in combined_dict
]

for num, file in enumerate(csvs_to_import):
    this_block_csv = pd.read_csv(os.path.join(blocks_path,file))
    axs[num].plot(this_block_csv)
    axs[num].set_ylabel('Acceleration [g] - ' + str(block_strings[num]))
    axs[num].set_title(rounded_list[num])
    

In [11]:
plt.savefig(os.path.join(
    project_path,
    'results',
    'accelerometer',
    'inspections',
    "".join(suptitle.split()),
), dpi = 250
)

In [12]:
file_name = "".join(suptitle.split()) + '_features.json'

# Combine the file name with the current directory to create the file path
file_path = os.path.join(
    project_path,
    'results',
    'accelerometer',
    'inspections')

# Serializing json
json_object = json.dumps(combined_dict, indent=4)
 
# Writing to sample.json
with open(os.path.join(file_path,file_name), "w") as outfile:
    outfile.write(json_object)

### Average all blocks for each condition within subject

In [15]:
json_dir = os.path.join(
    project_path,
    'results', 'accelerometer', 'inspections', 'Condition_1'
)
all_features_cond1 = pd.DataFrame()

# Loop through the JSON files in the directory
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(json_dir, filename)
        percept_id = filename[:6]

        # Load the JSON file
        with open(file_path, 'r') as f:
            json_data = json.load(f)
        
        # Initialize a dictionary to store the accumulated sums and counts for each key
        summed_values = {}
        count_values = {}
        
        # Iterate through the dictionaries in the JSON data
        for data_dict in json_data:
            # Accumulate the sums and counts for each key
            for key, value in data_dict.items():
                if key not in summed_values:
                    summed_values[key] = value
                    count_values[key] = 1
                else:
                    summed_values[key] += value
                    count_values[key] += 1
        
        # Calculate the averages for each key
        averaged_values = {key: summed_values[key] / count_values[key] for key in summed_values}
        averaged_values['Percept_ID'] = percept_id
        
        # Append the averaged values to the dataframe
        all_features_cond1 = all_features_cond1.append(averaged_values, ignore_index=True)




C:\Users\mathiopv\AppData\Local\Temp\ipykernel_18508\2685311361.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_features_cond1 = all_features_cond1.append(averaged_values, ignore_index=True)
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_18508\2685311361.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_features_cond1 = all_features_cond1.append(averaged_values, ignore_index=True)
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_18508\2685311361.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_features_cond1 = all_features_cond1.append(averaged_values, ignore_index=True)
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_18508\2685311361.py:37: FutureWarning: The frame.append method is deprecated and will 

In [16]:
# Print the resulting dataframe
all_features_cond1 = all_features_cond1.reindex(columns=['Percept_ID'] + list(all_features_cond1.columns[:-1]))
print(all_features_cond1)

  Percept_ID      freq  coefVar_intraTapInt  trace_RMSn  mean_raise_velocity  \
0     Sub005  2.766939             0.424583    0.045540             6.191110   
1     Sub006  2.962062             0.211093    0.029363             7.859107   
2     Sub009  3.041761             0.112239    0.036848             9.872084   
3     Sub014  2.703130             0.132782    0.129400            51.638945   
4     Sub021  2.777617             0.302006    0.144079            21.333090   
5     Sub025  2.929138             0.091372    0.165337            47.406006   
6     Sub028  2.402775             0.353027    0.456259            32.527386   
7     Sub029  2.765205             0.273971    0.154981            26.759368   
8     Sub033  1.439306             0.371660    0.105021            21.476653   
9     Sub065  3.395911             0.174063    0.179563            40.129215   

   coefVar_impactRMS  
0           0.118700  
1           0.099420  
2           0.125830  
3           0.196606  
4   

In [17]:
# Save the dataframe as JSON
json_file_path = os.path.join(json_dir, 'all_features_cond1.json')
all_features_cond1.to_json(json_file_path, orient='records')

### Plot all averaged values in three conditions

In [22]:
json_dir = os.path.join(
    project_path,
    'results', 'accelerometer', 'inspections\\'
)

feat_cond_1 = pd.read_json(str(json_dir) + 'all_features_cond1.json')
feat_cond_2 = pd.read_json(str(json_dir) + 'all_features_cond2.json')
feat_cond_3 = pd.read_json(str(json_dir) + 'all_features_cond3.json')

feat_cond_1

,Percept_ID,freq,coefVar_intraTapInt,trace_RMSn,mean_raise_velocity,coefVar_impactRMS
0,Sub005,2.766939,0.424583,0.045540,6.191110,0.118700
1,Sub006,2.962062,0.211093,0.029363,7.859107,0.099420
2,Sub009,3.041761,0.112239,0.036848,9.872084,0.125830
3,Sub014,2.703130,0.132782,0.129400,51.638945,0.196606
4,Sub021,2.777617,0.302006,0.144079,21.333090,0.185919
5,Sub025,2.929138,0.091372,0.165337,47.406006,0.098237
6,Sub028,2.402775,0.353027,0.456259,32.527386,0.236186
7,Sub029,2.765205,0.273971,0.154981,26.759368,0.149737
8,Sub033,1.439306,0.371660,0.105021,21.476653,0.337890
9,Sub065,3.395911,0.174063,0.179563,40.129215,0.074763


In [23]:
# Get the column names excluding 'Percept_ID'
columns = [col for col in feat_cond_1.columns if col != 'Percept_ID']

fig, axes = plt.subplots(1, len(columns), figsize=(16, 5))  # Adjust figsize as needed

# Iterate over each column and create a boxplot for each dataframe
for i, column in enumerate(columns):
    # Get the data for the column from each dataframe
    data1 = feat_cond_1[column]
    data2 = feat_cond_2[column]
    data3 = feat_cond_3[column]

     # Create a boxplot for the column in the corresponding subplot
    axes[i].boxplot([data1, data2, data3], showfliers=False)
    axes[i].scatter([1]*len(data1), data1, color='red')
    axes[i].scatter([2]*len(data2), data2, color='blue')
    axes[i].scatter([3]*len(data3), data3, color='green')
    axes[i].set_title(column)


    # Create a boxplot for the column in the corresponding subplot
    axes[i].boxplot([data1, data2, data3])
    axes[i].set_title(column)

# Add a legend at the bottom of the plot
legend_labels = ['M1S0', 'M1S1:Subh-Start', 'M1S1:Subh-Highest Amp']
legend_markers = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color) for color in ['red', 'blue', 'green']]
fig.legend(legend_markers, legend_labels, loc='lower center', ncol=3)

# Adjust spacing between subplots and legend
fig.tight_layout(rect=[0, 0.1, 1, 1])

In [24]:
plt.savefig(os.path.join(
    json_dir, 'All_avg_features_conds'
), dpi = 250
)